In [18]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np

import statsmodels.api as sm

# Purpose of the Notebook

Find the beta loadings of the ETFs per country

In [36]:
country_etfs = pd.read_csv('../Data/country_data/ETF_adj_close.csv',index_col=0)
country_etfs.index = pd.to_datetime(country_etfs.index)

In [37]:
# find monthly return - take last date and pct-change
country_returns = country_etfs.groupby(pd.Grouper(freq = 'M')).last().pct_change()
country_returns.index = country_returns.index.to_period('M')

# Fama French

In [16]:
ff5 = pdr.famafrench.FamaFrenchReader('Emerging_5_Factors',start = '2000-01-01').read()[0]

In [53]:
# lets store all betas for FF
betas_dict = {}
results_dict = {}
for c in country_returns:
    y = country_returns.loc[ff5.index][c]
    y = y.dropna()
    X = ff5.loc[y.index]
    X = sm.add_constant(X)
    model = sm.OLS(y, X)
    results = model.fit()
    betas_dict[c] = results.params
    results_dict[c] = results
    print(c+', '+y.index.min().strftime('%Y%m'))

Argentina, 201104
Brazil, 200008
Chile, 200712
China, 201105
Colombia, 201307
Egypt, 201003
Greece, 201201
India, 201203
Korea, 200006
Malaysia, 200001
Mexico, 200001
Pakistan, 201505
Peru, 200907
Philippines, 201010
Poland, 201006
Qatar, 201406
Russia, 201012
Saudi Arabia, 201510
South Africa, 200303
Taiwan, 200007
Thailand, 200805
Turkey, 200804
United Arab Emirates, 201406


In [55]:
pd.DataFrame(betas_dict)

,Argentina,Brazil,Chile,China,Colombia,Egypt,Greece,India,Korea,Malaysia,...,Philippines,Poland,Qatar,Russia,Saudi Arabia,South Africa,Taiwan,Thailand,Turkey,United Arab Emirates
const,0.001178,-0.006890,-0.007603,0.005089,-0.020948,-0.013269,-0.006090,-0.000227,-0.000598,-0.000763,...,-0.000650,-0.005593,-0.008544,-0.002458,0.000328,-0.002155,0.003362,0.004642,-0.012352,-0.008250
Mkt-RF,0.012105,0.013001,0.010709,0.008787,0.014921,0.011087,0.012114,0.009882,0.012716,0.006499,...,0.009497,0.012896,0.005184,0.010460,0.005110,0.011611,0.010109,0.010656,0.013434,0.006530
SMB,-0.001301,-0.010044,0.001400,-0.005435,0.003215,0.013303,0.001344,0.005731,-0.001461,0.001900,...,0.003259,0.000193,0.000821,-0.001532,0.009121,-0.006553,0.000273,0.002384,0.003156,0.004857
HML,0.003781,0.004788,-0.002309,0.002168,0.003225,-0.004867,0.008958,0.001641,-0.003035,-0.000501,...,-0.004428,-0.000268,0.004135,0.003312,0.001219,-0.002534,-0.005341,-0.003271,0.001106,0.007227
RMW,-0.001660,-0.005751,0.004691,0.002614,0.002382,-0.000075,-0.014572,0.005530,0.006137,-0.001184,...,0.002641,-0.003877,0.000391,-0.012602,-0.001346,0.001447,-0.000967,-0.003941,0.009217,-0.001062
CMA,0.003564,0.005566,0.013512,-0.012355,0.007212,-0.000795,-0.001990,-0.000312,0.001933,0.002646,...,0.007510,0.007993,0.002926,0.001455,0.004357,0.007942,0.000952,0.008595,0.009829,0.000226
RF,-0.022865,0.030875,0.007937,-0.028758,0.128181,0.109050,0.067758,0.009152,0.010695,0.011843,...,-0.005047,0.024654,0.081145,0.103595,0.058743,-0.013296,-0.014321,-0.031143,0.101000,0.018934
